In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna
from sklearn.model_selection import KFold
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')

In [ ]:
df_train.head(10)

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()
# no null values in training data


In [ ]:
# time is in object so we will convert it into datetime
df_train['time'] = pd.to_datetime(df_train['time'])
# df_train['time'] = pd.to_datetime(df_test['time'])
df_test['time'] = pd.to_datetime(df_test['time'])


In [ ]:
df_train.info()

In [ ]:
#  splitting the data
df_train['day_of_the_week']=df_train['time'].dt.dayofweek
df_train['hour']= df_train['time'].dt.hour
df_train['minute']= df_train['time'].dt.minute
df_train.drop('time', inplace=True, axis=1)

In [ ]:
df_train

In [ ]:
df_test['day_of_the_week'] = df_test['time'].dt.dayofweek
df_test['hour'] = df_test['time'].dt.hour
df_test['minute'] = df_test['time'].dt.minute
df_test.drop('time', inplace=True, axis=1)

In [ ]:
df_test

In [ ]:
# dropping day of the week,row id and congestion
row_id = df_test['row_id']
X = df_train.drop(['row_id', 'congestion', 'day_of_the_week'], axis=1)
Y = df_train['congestion']

In [ ]:
dict = {
    
    'NB' : 0,
    'EB' : 1,
    'NE' : 2,
    'NW' : 3,
    'SB' : 4,
    'SE' : 5,
    'SW' : 6,
    'WB' : 7
}

In [ ]:
X['direction'] = X['direction'].apply(lambda i : dict[i])

In [ ]:
X

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 1, 
                          learning_rate = 0.3,
                          max_depth = 15, 
                          alpha = 10, 
                          n_estimators = 100, 
                          verbose=1, 
                          min_child_weight = 1, 
                          colsample_bylevel = 1, 
                          reg_alpha = 2) 

xg_reg.fit(X_train, Y_train)


In [ ]:
# modifying test test
df_test['direction'] = df_test['direction'].apply(lambda i : dict[i])
df_test.drop(['row_id', 'day_of_the_week'], axis=1, inplace=True)

In [ ]:
pred = xg_reg.predict(df_test)

In [ ]:
pred

In [ ]:
output = pd.DataFrame({'row_id': row_id,'congestion': pred})

In [ ]:
output.to_csv('submission.csv', index=False)